In [2]:
# 1. Install and import DuckDB (one-time)
!pip install duckdb --quiet

# Import and set up
import duckdb
import pandas as pd

# Connect to an in-memory database
con = duckdb.connect(database=':memory:')

In [3]:
# 2. Register your cleaned CSV with DuckDB
# Path from /notebooks/ directory
clean_path = "../data_clean/online_retail_clean.csv"

# Register the CSV as a table called 'transactions'
con.execute(f"CREATE TABLE transactions AS SELECT * FROM read_csv_auto('{clean_path}')")

# Quick check
con.execute("DESCRIBE transactions").df()

,column_name,column_type,null,key,default,extra
0,InvoiceNo,VARCHAR,YES,None,None,None
1,StockCode,VARCHAR,YES,None,None,None
2,Description,VARCHAR,YES,None,None,None
3,Quantity,BIGINT,YES,None,None,None
4,InvoiceDate,TIMESTAMP,YES,None,None,None
5,UnitPrice,DOUBLE,YES,None,None,None
6,CustomerID,BIGINT,YES,None,None,None
7,Country,VARCHAR,YES,None,None,None
8,TotalPrice,DOUBLE,YES,None,None,None


In [4]:
# 3. Test a simple query first
con.execute("""
SELECT Country, COUNT(*) AS transaction_count, SUM(TotalPrice) AS revenue
FROM transactions
GROUP BY Country
ORDER BY revenue DESC
LIMIT 10
""").df()

,Country,transaction_count,revenue
0,United Kingdom,361845,6.700537e+06
1,Netherlands,2367,2.846615e+05
2,EIRE,7483,2.502852e+05
3,Germany,9493,2.216982e+05
4,France,8490,1.967128e+05
5,Australia,1256,1.370773e+05
6,Switzerland,1876,5.573940e+04
7,Spain,2532,5.477458e+04
8,Belgium,2069,4.091096e+04
9,Sweden,462,3.659591e+04


In [6]:
# 4. Full SQL Query for Customer-Level RFM Table
rfm_sql = """
WITH snapshot AS (
    SELECT MAX(InvoiceDate) AS snapshot_date FROM transactions
),
customer_metrics AS (
    SELECT
        CustomerID,
        MAX(InvoiceDate) AS last_purchase_date,
        COUNT(DISTINCT InvoiceNo) AS frequency,
        SUM(TotalPrice) AS monetary,
        MIN(InvoiceDate) AS first_purchase_date
    FROM transactions
    WHERE TotalPrice > 0  -- Extra safety
    GROUP BY CustomerID
)
SELECT
    cm.CustomerID,
    DATE_DIFF('day', cm.last_purchase_date, (SELECT snapshot_date FROM snapshot)) AS recency_days,
    cm.frequency,
    cm.monetary,
    DATE_DIFF('day', cm.first_purchase_date, cm.last_purchase_date) / 365.0 AS customer_lifespan_years,
    cm.first_purchase_date,
    cm.last_purchase_date,
    (SELECT snapshot_date FROM snapshot) AS analysis_date
FROM customer_metrics cm
ORDER BY monetary DESC
"""

# Execute and bring into pandas
rfm_df = con.execute(rfm_sql).df()

print(f"Customer-level table shape: {rfm_df.shape}")
print("\nTop 10 customers by monetary:")
rfm_df.head(10)

Customer-level table shape: (4337, 8)

Top 10 customers by monetary:


,CustomerID,recency_days,frequency,monetary,customer_lifespan_years,first_purchase_date,last_purchase_date,analysis_date
0,14646,1,73,280206.02,0.967123,2010-12-20 10:09:00,2011-12-08 12:12:00,2011-12-09 12:50:00
1,18102,0,60,259657.30,1.005479,2010-12-07 16:42:00,2011-12-09 11:50:00,2011-12-09 12:50:00
2,17450,8,46,187406.07,0.983562,2010-12-07 09:23:00,2011-12-01 13:29:00,2011-12-09 12:50:00
3,14911,1,201,143825.06,1.019178,2010-12-01 14:05:00,2011-12-08 15:54:00,2011-12-09 12:50:00
4,12415,24,21,124914.53,0.857534,2011-01-06 11:12:00,2011-11-15 14:22:00,2011-12-09 12:50:00
5,14156,9,55,117379.63,0.991781,2010-12-03 11:48:00,2011-11-30 10:54:00,2011-12-09 12:50:00
6,17511,2,31,91062.38,1.016438,2010-12-01 10:19:00,2011-12-07 10:12:00,2011-12-09 12:50:00
7,16029,38,62,72882.09,0.917808,2010-12-01 09:57:00,2011-11-01 10:27:00,2011-12-09 12:50:00
8,16684,4,28,66653.56,0.969863,2010-12-16 17:34:00,2011-12-05 14:06:00,2011-12-09 12:50:00
9,14096,4,17,65164.79,0.265753,2011-08-30 10:49:00,2011-12-05 17:17:00,2011-12-09 12:50:00


In [7]:
# 5. Save the RFM table
rfm_path = "../data_clean/customer_rfm.csv"
rfm_df.to_csv(rfm_path, index=False)
print(f"RFM table saved to {rfm_path}")

RFM table saved to ../data_clean/customer_rfm.csv
